In [ ]:
import pandas as pd
from transformers import pipeline
import google.generativeai as genai
from modules.SummaryManager import EvalSummaryManager

from modules.summarizer import Summarizer
import evaluate

In [ ]:
data = pd.read_csv('./sample_data/discharge.csv')

In [ ]:
summarizer = pipeline("summarization", model="Falconsai/medical_summarization")


In [ ]:
text = data.iloc[0]['text']

print(summarizer(text, max_length=2000, min_length=1500, do_sample=False)[0]['summary_text'])

In [ ]:
summarizer = Summarizer(model='gemini')

discharge = '''
    Patient name: John Smith
    DOB: 03/15/1972
    MRN: 12345678
    Admitted: 05/30/2025
    Discharged: 06/04/2025
    Admitting & Discharging Physician: Dr. Emily Thompson

    Patient was admitted with a 3-day history of productive cough, fever up to 39°C, shortness of breath, and pleuritic chest pain. Chest X-ray showed right lower lobe pneumonia. Started on IV ceftriaxone and azithromycin. Given oxygen via nasal cannula to keep saturation above 92%.

    Fever resolved after 3 days. WBC dropped from 14.2 to 8.6. Cultures were negative. Switched to oral amoxicillin-clavulanate on hospital day 4. Blood glucose managed with insulin sliding scale. BP controlled with amlodipine.

    Discharge meds:

        Amoxicillin-clavulanate 875/125 mg PO BID for 3 more days

        Amlodipine 5 mg daily

        Metformin 500 mg BID

        Acetaminophen 500 mg PRN

    Follow-up with primary care in 1 week. Repeat chest X-ray in 6 weeks. Endocrinology referral for diabetes.

    Patient discharged stable, afebrile, eating and walking independently.
'''

summ = summarizer.summ(discharge)

In [ ]:


rouge = evaluate.load('rouge')

results = rouge.compute(predictions=[summ], references=[discharge])

for k, v in results.items():
    print(f'{k}: {v:.4f}')

In [ ]:
bs = evaluate.load('bertscore')
results = bs.compute(predictions=[summ], references=[discharge], lang="en")


In [ ]:
results

In [ ]:
results

In [ ]:
texts = [
    'The patient is a 67-year-old male with a history of congestive heart failure, hypertension, and type 2 diabetes mellitus. He was admitted with worsening shortness of breath and leg swelling. Echocardiogram showed an ejection fraction of 35%. He was started on intravenous diuretics and his symptoms improved. Blood glucose levels were managed with insulin. The patient was counseled on a low-sodium, heart-healthy diet and discharged on Lasix, Metoprolol, and Lisinopril. Follow-up was scheduled with cardiology and endocrinology. He was advised to monitor weight daily and return if symptoms worsen.',
    'Artificial intelligence is reshaping industries worldwide. Companies are investing heavily in AI tools to optimize operations, enhance customer service, and develop new products. From healthcare to finance, AI applications are improving diagnostics, fraud detection, and decision-making. However, concerns remain about job displacement, ethical AI use, and data privacy. Governments are beginning to propose regulatory frameworks to ensure safe development and deployment. Experts emphasize the importance of transparency, accountability, and fairness in AI systems. The global AI market is expected to grow significantly over the next decade, driving innovation while presenting complex challenges for society.',
    'The Industrial Revolution marked a major turning point in history, transforming economies from agrarian to industrial. Beginning in the late 18th century in Britain, it saw the invention of machines such as the spinning jenny and steam engine. These innovations revolutionized textile production and transportation. Urbanization increased as people moved to cities for factory jobs. While industrialization boosted productivity and wealth, it also brought harsh labor conditions and environmental degradation. Social reform movements emerged to address these issues, laying the groundwork for modern labor laws. The revolution eventually spread globally, influencing economic structures and societal organization.',
    'In recent weeks, extreme weather events have impacted multiple countries. Flooding in southern Asia has displaced thousands, while record-breaking heat waves in parts of Europe and North America have strained power grids and healthcare systems. Scientists attribute these patterns to climate change, warning of increasingly frequent and intense events in the future. Governments are responding with emergency aid, infrastructure reviews, and climate adaptation plans. Environmental groups are urging faster action on emissions reductions. These events underscore the urgent need for coordinated international efforts to mitigate climate risks and protect vulnerable populations worldwide.',
    'Photosynthesis is a critical biological process through which green plants, algae, and some bacteria convert light energy into chemical energy. Using sunlight, carbon dioxide from the air, and water from the soil, plants produce glucose and oxygen. This process occurs primarily in the chloroplasts of plant cells and involves two main stages: the light-dependent reactions and the Calvin cycle. Photosynthesis not only sustains plant growth but also forms the foundation of most food chains. Additionally, it plays a vital role in regulating atmospheric carbon dioxide levels, contributing to climate stability and supporting life on Earth.',
]

In [ ]:
discharges = data[:25]['text'].tolist()

In [ ]:
eval = EvalSummaryManager()
eval.add_model('bart')
eval.add_model('t5')
eval.add_model('gemini')
results = eval.everything_eval(discharges)
results

In [ ]:
results.to_csv('output_metricas_mimic.csv', index=False)

In [ ]:
results

In [ ]:
import google.generativeai as genai
import json

with open('config/cred.json') as f:
    config = json.load(f)
    api_key = config['api_key']

genai.configure(api_key=api_key)

model = genai.GenerativeModel(model_name='gemini-2.5-flash')

response = model.generate_content('Explain how AI works in a few words')
print(response.text)


In [ ]:
prompt = '''

summarize the following text:
The patient is a 67-year-old male with a history of congestive heart failure, hypertension, and type 2 diabetes mellitus. He was admitted with worsening shortness of breath and leg swelling. Echocardiogram showed an ejection fraction of 35%. He was started on intravenous diuretics and his symptoms improved. Blood glucose levels were managed with insulin. The patient was counseled on a low-sodium, heart-healthy diet and discharged on Lasix, Metoprolol, and Lisinopril. Follow-up was scheduled with cardiology and endocrinology. He was advised to monitor weight daily and return if symptoms worsen.
'''


model.generate_content(prompt).text

In [ ]:
from google.cloud import bigquery

# Path to your service account key file
key_path = "mimic-test-key.json"

# Create a client using the key file
client = bigquery.Client.from_service_account_json(key_path)

# Your SQL query
query = """
SELECT *
FROM `physionet-data.mimiciv_note.discharge`
LIMIT 10
"""

# Run the query
query_job = client.query(query)

# Convert to DataFrame
df = query_job.to_dataframe()

print(df.head())


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='mimic-test-465504')

df = client.query("""
SELECT *
FROM `physionet-data.mimiciv_note.discharge`
LIMIT 10
""").to_dataframe()

print(df)


In [ ]:
discharge = '''


'''

In [ ]:
prompt = 'Summarize the following text, return just a plain text with no formatting other then line break elements: \n' + discharge

In [ ]:
genai.configure(api_key="")

model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content(prompt)

print(response.text)
